In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
# !wget http://ipsar.fesb.unist.hr/ipsar_data/heridal.zip
# !unzip ./heridal.zip

In [4]:
!ls ../heridal

patches  testImages  trainImages


In [5]:
from drone_sar.ipsar_dataset import IPSARDataset
from drone_sar.lightning_detector import get_lightning_trainer, LightningDetector
from collections import Counter

In [6]:
train_data = IPSARDataset("../heridal/trainImages/")
test_data = IPSARDataset("../heridal/testImages")
item = test_data[0]

In [7]:
len(train_data)

1583

In [8]:
# [it["boxes"] for it in train_data]

In [9]:
# Counter([len(el["boxes"]) for el in train_data])

In [10]:
item.keys()

dict_keys(['img_path', 'pil', 'target_sizes', 'boxes', 'class_labels'])

In [11]:
item["boxes"].shape, item["class_labels"].shape

(torch.Size([2, 4]), torch.Size([2]))

In [12]:
# IPSARDataset.plt_show(item, size=10)

In [13]:
lightning_detector = LightningDetector(lr=5e-6)

/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/transformers/models/detr/image_processing_detr.py:776: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(
Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
trainer = get_lightning_trainer("IPSAR-DETR", max_epochs=1000)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
train_dl = train_data.get_dataloader(lightning_detector.processor, bs=10, shuffle=True)
test_dl = test_data.get_dataloader(lightning_detector.processor, bs=10, shuffle=False)

In [16]:
batch, info = next(iter(train_dl))

In [17]:
import torch
torch.isinf

<function torch._VariableFunctionsClass.isinf>

In [18]:
len(batch["labels"][0]['boxes'])

0

In [19]:
from tqdm.auto import tqdm

# [torch.isinf(b["boxes"]).all() for batch, _ in tqdm(test_dl) for b in batch["labels"] if len(b["boxes"]) > 0]

In [ ]:
trainer.fit(lightning_detector, train_dataloaders=train_dl, val_dataloaders=test_dl)

/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/iz/workspace/DroneSearchAndRescue/notebooks/.checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                   | Params
-------------------------------------------------
0 | model | DetrForObjectDetection | 41.5 M
-------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.037   Total estimated model params size (MB)


Sanity Checking: |                                                | 0/? [00:00<?, ?it/s]

/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLd

Training: |                                                       | 0/? [00:00<?, ?it/s]

Validation: |                                                     | 0/? [00:00<?, ?it/s]

...

Validation: |                                                     | 0/? [00:00<?, ?it/s]

Validation: |                                                     | 0/? [00:00<?, ?it/s]

Validation: |                                                     | 0/? [00:00<?, ?it/s]

Validation: |                                                     | 0/? [00:00<?, ?it/s]

Validation: |                                                     | 0/? [00:00<?, ?it/s]

In [ ]:
outputs_train = trainer.predict(lightning_detector, train_dl)

In [ ]:
outputs_val = trainer.predict(lightning_detector, test_dl)

In [ ]:
# !rm -rf /home/iz/.cache/joblib/